In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:

import pandas as pd
import numpy as np
import torch
import torchio as tio
from matplotlib import pyplot as plt
from pathlib import Path
from tqdm.autonotebook import tqdm
import zarr
from nibabel import ecat

from data_loading.data_set import EcatDFDataset, get_default_tio_transform

In [36]:
path_df = Path('/home/fabian/projects/phd/ai_radiation_therapy/ai_head_neck_tumor/data/clinical_data_has_CT_PT_21feb23.feather')
df = pd.read_feather(path_df)
data_path = Path(df.iloc[0].blue_data_path)



In [104]:
ds = EcatDFDataset(df, mode='full', cv_split=0,  img_columns=['CT_img_path'])
x, y = ds[10]
print(x.shape, x.min(), x.max())

# shapes = []
# for i in tqdm(range(len(ds))):
#     x, y = ds[i]
#     shapes.append(x.shape)
   
# shapes = [tuple(s) for s in shapes] 
# df['shapes'] = shapes
# df.to_feather(path_df)

# unequals = []
# for i in tqdm(range(len(ds))):
#     x, y = ds[i]
#     if not (x[1] == x[3]).all():
#         print(i, 'not equal')
#         unequals.append(i)

torch.Size([1, 128, 128, 140]) tensor(-999.8713, dtype=torch.float64) tensor(2976.0000, dtype=torch.float64)


# Save ecat as zarr arrays

In [79]:
store = zarr.ZipStore(data_path.parent / 'cnn_data.zarr.zip', mode='w')
# Create a Zarr group to hold the data
root = zarr.group(store=store, overwrite=True)

In [40]:
def read_ecat(img_path):
    img = ecat.load(img_path)
    data = np.transpose(img.get_fdata(), [3, 0, 1, 2])
    return data

In [80]:
store = zarr.ZipStore('/home/fabian/projects/phd/ai_radiation_therapy/data_preprocessed/cnn_data.zarr.zip', mode='w')
# Create a Zarr group to hold the data
root = zarr.group(store=store, overwrite=True)

for idx, row in tqdm(df.iterrows(), total=len(df)):
    for col in ['CT_img_path', 'CT_roi_path', 'PT_img_path', 'PT_roi_path']:
        img_name = row[col]
        # print(data_path/ img_name)
        data = read_ecat(data_path / img_name)
        chunk_size = (1, 32, 32, data.shape[-1])
        root.array(img_name, data=data, chunks=chunk_size, compressor=zarr.Blosc(cname='zstd', clevel=3, shuffle=2), order='C')
        
store.close()

  0%|          | 0/1035 [00:00<?, ?it/s]

In [97]:
test_root = zarr.open('/home/fabian/projects/phd/ai_radiation_therapy/data_preprocessed/cnn_data.zarr.zip')
test = np.array(test_root[df['CT_img_path'].iloc[10]])

# Examine transforms

In [109]:
transform = tio.Compose([
        tio.RescaleIntensity(out_min_max=(0, 255), percentiles=(0.5, 99.5)),
        tio.CropOrPad((128, 128, 140)),
        tio.RandomAffine(scales=0.1, degrees=10),
        tio.RandomElasticDeformation()
        ])